# ZDO 2022 - Pig Leg Surgery

Navrhněte a naprogramujte aplikaci pro detekci chirurgických nástrojů v čase
Vstupem jsou cesty k videím z pohledu shora během šití prasečí nožičky
Cílem je určit `[x, y]` pozice obarvených hrotů chirurgických nástrojů pro každý snímek
Výstup (bude sloužit pro vyhodnocení na našich datech):
json pro každou sekvenci obrázků bude ve formátu: 

```python
annotation={
    "filename": ["vid1.mp4", "vid1.mp4", "vid1.mp4"...], # pth.parts[-1]
    "frame_id": [0,1,1,...],
    "object_id": [0,0,1],
    "x_px": [110, 110, 300], # x pozice obarvených hrotů v pixelech
    "y_px": [50, 50, 400],   # y pozice obarvených hrotů v pixelech
    "annotation_timestamp": [],
}
[{
  "file_name": "<název_obrázku>.jpg",
  "predictions": [{
   "center": [int(x), int(y)],   # x a y pozice obarvených hrotů v pixelech
    "obj_id": str,    # id objektu
  }, …]
}, …]
```


vykreslení výstupů do videa:

centry hrotů vykreslit buď viditelným bodem nebo ohraničujícím boxem rozumné velikostí (např. 100x100 px)
* každý objekt označen jinou barvou
* Primární úloha: tracking needle holderu
* Bonusová úloha: tracking ostatních nástrojů
 

# Annotation tool CVAT


http://147.228.124.52:8080/

Popis práce s CVAT naleznete na [stránkách loňského zadání](https://nbviewer.org/github/mjirik/ZDO/blob/master/ZDOsem2021.ipynb)

Z něj si stáhnete anotce ve formátu CVAT for videos

# Práce s videem a anotacemi

Základy práce s videem lze načerpat [zde](https://nbviewer.org/github/mjirik/ZDO/blob/master/ZDOsem2020.ipynb)

Převod anotací z formátu CVAT for videos 

In [35]:
from pathlib import Path
import lxml
from lxml import etree

pth = Path("sem/2022/annotations.xml")
# root = etree.fromstring(xml, base_url=pth)

annotation={
    "filename": [], # pth.parts[-1]
    "frame_id": [],
    "object_id": [],
    "x_px": [], # x pozice obarvených hrotů v pixelech
    "y_px": [],   # y pozice obarvených hrotů v pixelech
    "annotation_timestamp": [],
}


tree = etree.parse(pth)

updated = tree.xpath("//updated")[0].text # date of last change in CVAT

for track in tree.xpath('track'):
    for point in track.xpath("points"):
        pts = point.get("points").split(",")
        x, y = pts
        annotation["filename"].append(str(pth))
        annotation["object_id"].append(track.get("id"))
        annotation["x_px"].append(x)
        annotation["y_px"].append(y)
        annotation["frame_id"].append(point.get("frame"))
        annotation["annotation_timestamp"].append(updated)
        
    # print(track.find('./@id'))
    


In [37]:
import pandas as pd
pd.DataFrame(annotation)

,filename,frame_id,object_id,x_px,y_px,annotation_timestamp
0,sem\2022\annotations.xml,47,0,3814.91,1522.37,2022-03-11 07:39:11.353911+00:00
1,sem\2022\annotations.xml,48,0,3757.17,1485.89,2022-03-11 07:39:11.353911+00:00
2,sem\2022\annotations.xml,49,0,3699.43,1449.41,2022-03-11 07:39:11.353911+00:00
3,sem\2022\annotations.xml,50,0,3641.68,1412.94,2022-03-11 07:39:11.353911+00:00
4,sem\2022\annotations.xml,51,0,3583.94,1376.46,2022-03-11 07:39:11.353911+00:00
...,...,...,...,...,...,...
950,sem\2022\annotations.xml,496,1,1389.90,204.80,2022-03-11 07:39:11.353911+00:00
951,sem\2022\annotations.xml,497,1,1389.90,204.80,2022-03-11 07:39:11.353911+00:00
952,sem\2022\annotations.xml,498,1,1389.90,204.80,2022-03-11 07:39:11.353911+00:00
953,sem\2022\annotations.xml,499,1,1389.90,204.80,2022-03-11 07:39:11.353911+00:00


# Odevzdání

* ve formě git repozitáře
* dokumentace na gitu (jména autorů ideálně v readme.md)
* spustitelný automatický test - viz [šablona, kterou můžete využít](https://github.com/mjirik/ZDO2022)